# Trigeorgis binomial tree for European call options pricing

When it comes to finding the price of a European option, the easiest way of proceeding is probably the Black-Scholes formula. However, with other types of derivatives, such as American options or other exotic products, there is no analytical way to calculate the price, and we must rely on other methods. One of them is running a Monte Carlo simulation and another is through binomial trees. In this little piece of code, we are computing the price of a European call through a **Trigeorgis tree with the JR approach**.

**Binomial trees** let us simulate possible future prices of the option's underlying stock at maturity and then, by discounting the expected payoff of the product through a backward procedure, we can obtain the present value of the option. The tricky point of this method is how we obtain the probabilities of the market going up or down in each time step in such a way that is consistent with the Black-Scholes model. There are several approaches to this, being the **Cox-Ross-Rubinstein (1979)** and the **Jarrow-Rud (1983)** the most known models.

The **Trigeorgis binomial model (1992)** is a method of recombining binomial trees that matches well with the assumptions of Black-Scholes, by assuming that the price of the underlying follows a lognormal distribution, and is also compatible with the CRR and JR approaches. This makes the tree additive instead of multiplicative and simplifies the calculus.

Let's denote the current spot price as $S_{00}$. At each node, the spot price of the underlying is:

$$S_{ij}=Su^{j}d^{i-j}$$

The *u* and *d* values, as well as $p_u$ and $p_d$ (probabilities of the price going up or down), under the JR approach in a Trigerogis tree are given by:

$$\Delta x_u = \sqrt{\sigma^2\Delta t + \nu^2\Delta t^2}$$
$$\Delta x_d = -\Delta x_u$$
$$\nu=r-\frac{\sigma^2}{2}$$
$$u=e^{\Delta x_u}$$
$$d=e^{\Delta x_d}$$
$$p_u=\frac{1}{2}+\frac{\nu \Delta t}{2\Delta x_u}$$
$$p_d=1-p_u$$

In [1]:
import numpy as np
from math import sqrt,exp

In [2]:
def trigTree_EuCall(K,T,S,sig,r,N):
    dt=T/N
    nu=r-0.5*sig**2
    dxu=sqrt(sig**2*dt+nu**2*dt**2)
    dxd=-dxu
    u=exp(dxu)
    d=exp(dxd)
    pu=0.5+0.5*nu*dt/dxu
    pd=1-pu
    disc=exp(-r*dt)
    
    # Tree initialization
    St=np.zeros((N+1,1))
    for x in range(0,N+1):
        St[x,0]=S*exp(N*dxd)
    for y in range(1,N+1):
        St[y,0]=St[y-1,0]*u/d
    
    # Payoff
    C=np.zeros((N+1,1))
    for z in range(0,N+1):
        C[z,0]=max(St[z,0]-K,0)

    for i in range(N,0,-1):
        for j in range(0,i):
            C[j,0]=disc*(pu*C[j+1,0]+pd*C[j,0])
    return C[0,0]

In [3]:
K=100
T=1
S=100
sig=0.2
r=0.05
N=1000

callPrice=trigTree_EuCall(K,T,S,sig,r,N)
print(f'Call price: {round(callPrice, 5)}')

Call price: 10.44863
